In [4]:
import yfinance as yf
import pandas as pd
# Set pandas display options to prevent column concatenation in print
pd.set_option('display.max_rows', None)
pd.set_option('display.expand_frame_repr', False)

pairs = [
    ['NVDA', 'AMD'],   # NVIDIA / Advanced Micro Devices
    ['MSFT', 'GOOGL'], # Microsoft / Alphabet (Google)
    ['AAPL', 'MSFT'],  # Apple / Microsoft
    ['V', 'MA'],       # Visa / Mastercard
    ['CRM', 'ADBE'],   # Salesforce / Adobe
    ['INTC', 'QCOM'],  # Intel / Qualcomm
    ['CSCO', 'ANET'],  # Cisco / Arista Networks
    ['ORCL', 'SAP'],   # Oracle / SAP
    ['UBER', 'LYFT'],  # Uber / Lyft
    ['META', 'SNAP']   # Meta Platforms / Snap Inc.
]
tickers = [ticker for pair in pairs for ticker in pair]
print("loading data...")
data = yf.download(tickers, start='2020-01-01', end='2025-01-01')
print(data)
# Analyze NaNs in each column
nan_counts = data.isna().sum()
print("Number of NaNs in each column:")
print(nan_counts)
#0 NaNs, no data cleaning needed




/var/folders/31/_t7r4pl15ng3lrdcglpq152w0000gr/T/ipykernel_11449/727098257.py:21: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(tickers, start='2020-01-01', end='2025-01-01')
[**********************63%*****                  ]  12 of 19 completed

loading data...


[*********************100%***********************]  19 of 19 completed


Price            Close                                                                                                           ...    Volume                                                                                               
Ticker            AAPL        ADBE         AMD        ANET         CRM       CSCO       GOOGL       INTC       LYFT          MA  ...        MA       META      MSFT        NVDA      ORCL      QCOM       SAP       SNAP       UBER         V
Date                                                                                                                             ...                                                                                                         
2020-01-02   72.538513  334.429993   49.099998   12.795000  165.251709  40.688911   67.965225  53.666470  43.580002  293.338531  ...   3272400   12077100  22622100   237536000  13899600   8413900   2967500   26062600   20578900   8733000
2020-01-03   71.833290  331.809998   48.599998  